<a href="https://colab.research.google.com/github/Fsrkh/Persian-dates-NLP/blob/main/date_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow_text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 859.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 82.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 re

In [1]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text

In [2]:
data = pd.read_csv("dates.csv")
data.head()

,Formal Date,Informal Date 1,Informal Date 2,Informal Date 3,Informal Date 4,Informal Date 5,Informal Date 6,Informal Date 7,Informal Date 8,Informal Date 9
0,1400/01/01,اول فروردین هزار و چهارصد,یک فروردین هزار و چهارصد,یک فروردین ۱۴۰۰,فروردین یک ۱۴۰۰,اول فروردین ۱۴۰۰,یکِ فروردین ۱۴۰۰,یکم فروردین ۱۴۰۰,اول فروردین ماه ۱۴۰۰,فروردینِ یک ۱۴۰۰
1,1400/02/10,دهم اردیبهشت هزار و چهارصد,ده اردیبهشت هزار و چهارصد,ده اردیبهشت ۱۴۰۰,اردیبهشت ده ۱۴۰۰,دهم اردیبهشت ۱۴۰۰,دهِ اردیبهشت ۱۴۰۰,دهم اردیبهشت ماه ۱۴۰۰,اردیبهشتِ ده ۱۴۰۰,ده اردیبهشت ۱۴۰۰
2,1401/03/05,پنجم خرداد هزار و چهارصد و یک,پنج خرداد هزار و چهارصد و یک,پنج خرداد ۱۴۰۱,خرداد پنج ۱۴۰۱,پنجم خرداد ۱۴۰۱,پنجِ خرداد ۱۴۰۱,پنجم خرداد ماه ۱۴۰۱,خردادِ پنج ۱۴۰۱,پنج خرداد ۱۴۰۱
3,1401/04/15,پانزدهم تیر هزار و چهارصد و یک,پانزده تیر هزار و چهارصد و یک,پانزده تیر ۱۴۰۱,تیر پانزده ۱۴۰۱,پانزدهم تیر ۱۴۰۱,پانزدهِ تیر ۱۴۰۱,پانزدهم تیر ماه ۱۴۰۱,تیرِ پانزده ۱۴۰۱,پانزده تیر ۱۴۰۱
4,1402/05/20,بیستم مرداد هزار و چهارصد و دو,بیست مرداد هزار و چهارصد و دو,بیست مرداد ۱۴۰۲,مرداد بیست ۱۴۰۲,بیستم مرداد ۱۴۰۲,بیستِ مرداد ۱۴۰۲,بیستم مرداد ماه ۱۴۰۲,مردادِ بیست ۱۴۰۲,بیست مرداد ۱۴۰۲


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Formal Date      761 non-null    object
 1   Informal Date 1  761 non-null    object
 2   Informal Date 2  761 non-null    object
 3   Informal Date 3  761 non-null    object
 4   Informal Date 4  761 non-null    object
 5   Informal Date 5  761 non-null    object
 6   Informal Date 6  138 non-null    object
 7   Informal Date 7  138 non-null    object
 8   Informal Date 8  138 non-null    object
 9   Informal Date 9  138 non-null    object
dtypes: object(10)
memory usage: 59.6+ KB


In [8]:
context_raw = data.iloc[:, 1:].astype(str).apply(lambda row: ' '.join(row.values), axis=1).to_numpy()
target_raw = data.iloc[:, 0].astype(str).to_numpy()

In [9]:
BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 64

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

In [10]:
for example_context_strings, example_target_strings in train_raw.take(1):
    print(example_context_strings[:5])
    print()
    print(example_target_strings[:5])
    break

tf.Tensor(
[b'\xd8\xaf\xd9\x87\xd9\x85 \xd9\x85\xd9\x87\xd8\xb1 \xd9\x87\xd8\xb2\xd8\xa7\xd8\xb1 \xd9\x88 \xd8\xaf\xd9\x88\xdb\x8c\xd8\xb3\xd8\xaa \xd9\x88 \xd8\xb4\xd8\xb3\xd8\xaa \xd8\xaf\xd9\x87 \xd9\x85\xd9\x87\xd8\xb1 \xdb\xb1\xdb\xb2\xdb\xb6\xdb\xb0 \xd9\x85\xd9\x87\xd8\xb1 \xd8\xaf\xd9\x87 \xdb\xb1\xdb\xb2\xdb\xb6\xdb\xb0 \xd8\xaf\xd9\x87\xd9\x85 \xd9\x85\xd9\x87\xd8\xb1 \xdb\xb1\xdb\xb2\xdb\xb6\xdb\xb0 \xd9\x85\xd9\x87\xd8\xb1 \xd8\xaf\xd9\x87 \xdb\xb1\xdb\xb2\xdb\xb6\xdb\xb0 nan nan nan nan'
 b'\xd8\xa8\xdb\x8c\xd8\xb3\xd8\xaa \xd9\x88 \xd8\xaf\xd9\x88\xd9\x85 \xd9\x85\xd9\x87\xd8\xb1 \xd9\x87\xd8\xb2\xd8\xa7\xd8\xb1 \xd9\x88 \xda\x86\xd9\x87\xd8\xa7\xd8\xb1\xd8\xb5\xd8\xaf \xd9\x88 \xda\x86\xd9\x87\xd8\xa7\xd8\xb1 \xd8\xa8\xdb\x8c\xd8\xb3\xd8\xaa \xd9\x88 \xd8\xaf\xd9\x88 \xd9\x85\xd9\x87\xd8\xb1 \xd9\x87\xd8\xb2\xd8\xa7\xd8\xb1 \xd9\x88 \xda\x86\xd9\x87\xd8\xa7\xd8\xb1\xd8\xb5\xd8\xaf \xd9\x88 \xda\x86\xd9\x87\xd8\xa7\xd8\xb1 \xd8\xa8\xdb\x8c\xd8\xb3\xd8\xaa \xd9\x88 \xd8\xa